In [1]:
import numpy as np
import tensorflow as tf
from random import randint
import scipy as scipy
import scipy.interpolate as interpolate
import math
from random import *
from sklearn.decomposition import PCA
from sklearn.datasets import load_boston
from scipy.stats import logistic
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt2
from keras_radam.training import RAdamOptimizer

import tflib as lib
import tflib.ops.linear
import tflib.ops.conv2d
import tflib.ops.batchnorm
import tflib.ops.deconv2d
import tflib.save_images
import tflib.plot


Lambda = 10.0
BATCH_SIZE = 38
BATCH_SIZE_2 = 100
dimensionality = 13
num_epochs = 80
num_iters = 1000
k=2
max_grad_norm = 1000
gamma = 1.0

# main parameters

C=10.0

# additional parameters
sigma=0.8

number_of_neurons_regr = 200
number_of_points = 506-50
number_of_neurons = number_of_points

/home/rust/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/rust/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/rust/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/rust/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:5

In [2]:
boston = load_boston(return_X_y=False)
print(np.size(np.array(boston['target'])))
data = []
target = []
rr = np.random.permutation(506)
for x in rr:
    data.append(boston['data'][x])
    target.append(boston['target'][x])
# training data
_train_x = np.array(data)
_train_y = np.array(target)
print(_train_x.shape)
print(_train_y.shape)
print(np.max(_train_y))
rho = 0.5
var = np.var(_train_y)
print(rho)

506
(506, 13)
(506,)
50.0
0.5


In [3]:
new_lr = tf.placeholder(tf.float32, shape=[], name="new_learning_rate")
Dataplace = tf.placeholder(tf.float32, shape=(dimensionality, number_of_points))
Data = tf.placeholder(tf.float32, shape=(BATCH_SIZE, dimensionality))
outputs = tf.placeholder(tf.float32, shape=(BATCH_SIZE,1)) 

tf_data_x = gamma*tf.random_normal([BATCH_SIZE, dimensionality]) # аргументы функции
tf_data_y = tf.reduce_mean(tf.math.cos(tf.matmul(tf_data_x, Dataplace)), axis=1) # значения функции

tf_data_w = tf.placeholder(tf.float32, shape=(number_of_neurons,1))
tf_data_B = tf.placeholder(tf.float32, shape=(dimensionality,number_of_neurons))
tf_data_P = tf.placeholder(tf.float32, shape=(dimensionality, dimensionality))

tf_w_regr = tf.placeholder(tf.float32, shape=(number_of_neurons_regr,1))
tf_B_regr = tf.placeholder(tf.float32, shape=(dimensionality,number_of_neurons_regr))
tf_bias_regr = tf.placeholder(tf.float32, shape=(number_of_neurons_regr))


tf_data_second = gamma*tf.random_normal([BATCH_SIZE_2, dimensionality])
Lbd = tf.placeholder(tf.float32, shape=[], name="lambda")

# characteristic function parameters
w = tf.Variable(tf.random_normal([number_of_neurons,1], stddev=0.0), name="neuron_weights")
B = tf.Variable(initial_value=Dataplace, name="weights")
# regression function parameters
w_regr = tf.Variable(tf.random_normal([number_of_neurons_regr,1], stddev=0.35), name="neuron_weights")
B_regr = tf.Variable(tf.random_normal([dimensionality,number_of_neurons_regr], stddev=0.35), name="weights")
bias_regr = tf.Variable(tf.random_normal([number_of_neurons_regr], stddev=0.0), name="biases")

prediction = (2/number_of_neurons)*tf.matmul(tf.math.cos(tf.matmul(tf_data_x, B)), tf.nn.sigmoid(w))
penalty = tf.square((2/number_of_neurons)*tf.reduce_sum(tf.nn.sigmoid(w))-1.0)

out_loss = (1-rho)*tf.reduce_mean(tf.square(prediction - tf_data_y)) + C*penalty

prediction2 = (2/number_of_neurons)*tf.matmul(tf.math.cos(tf.matmul(tf_data_second, B)), tf.nn.sigmoid(w))
prediction_regr2 = tf.matmul(tf.nn.tanh(tf.nn.bias_add(tf.matmul(tf_data_second, B_regr), bias_regr)), w_regr)

grad_psi = tf.reshape(tf.gradients(prediction2, [tf_data_second])[0], [BATCH_SIZE_2, dimensionality])
grad_psi_regr = tf.reshape(tf.gradients(prediction_regr2, [tf_data_second])[0], [BATCH_SIZE_2, dimensionality])

tf_prediction2 = (2/number_of_neurons)*tf.matmul(tf.math.cos(tf.matmul(tf_data_second, tf_data_B)), tf.nn.sigmoid(tf_data_w))
tf_prediction_regr2 = tf.matmul(tf.nn.tanh(tf.nn.bias_add(tf.matmul(tf_data_second, tf_B_regr), tf_bias_regr)), tf_w_regr)

tf_data_grad_psi = tf.reshape(tf.gradients(tf_prediction2, [tf_data_second])[0], [BATCH_SIZE_2, dimensionality])
tf_data_grad_psi_regr = tf.reshape(tf.gradients(tf_prediction_regr2, [tf_data_second])[0], [BATCH_SIZE_2, dimensionality])

old_part = tf.matmul(tf_data_grad_psi, tf_data_P)
old_part_regr = tf.matmul(tf_data_grad_psi_regr, tf_data_P)

loss = out_loss + Lbd*(1-rho)*tf.reduce_mean(tf.reduce_sum(tf.square(tf.subtract(grad_psi, old_part)), axis=1)) + \
       Lbd*rho*tf.reduce_mean(tf.reduce_sum(tf.square(tf.subtract(grad_psi_regr, old_part_regr)), axis=1))/var

#offset = tf.random.uniform(shape=[], minval=0, maxval=12, dtype=tf.int32)*BATCH_SIZE
x_plus_error = Data+sigma*tf.random_normal([BATCH_SIZE, dimensionality])
regression = tf.matmul(tf.nn.tanh(tf.nn.bias_add(tf.matmul(x_plus_error, B_regr), bias_regr)), w_regr)
sdr = tf.reduce_mean(tf.square(regression-outputs))

loss_hybrid = loss + rho*sdr/var

In [4]:
target = tf.train.AdamOptimizer(learning_rate=1e-4, beta1=0.5, beta2=0.9).minimize(loss_hybrid)

In [5]:
def sigmoid(x):
  return 1. /(1+np.exp(-x))

In [6]:
def euclidean_distance(pnt1, pnt2):
    '''Finds the distance between 2 points: pnt1, pnt2'''
    # element-wise computations are automatically handled by numpy
    return sum((pnt1 - pnt2) ** 2)

from collections import defaultdict

def find_majority(labels):
    '''Finds the majority class/label out of the given labels'''
    # defaultdict(type) is to automatically add new keys without throwing error.
    counter = defaultdict(int)
    for label in labels:
        counter[label] += 1

    # Finding the majority class.
    majority_count = max(counter.values())
    for key, value in counter.items():
        if value == majority_count:
            return key

def new_predict(k, train_pnts, train_labels, test_pnts):
    '''
    Predicts the new data-point's category/label by 
    looking at all other training labels
    '''
    # distances contains tuples of (distance, label)
    distances = [(euclidean_distance(test_pnts, pnt), label)
                    for (pnt, label) in zip(train_pnts, train_labels)]
    # sort the distances list by distances
    compare = lambda distance: distance[0]
    by_distances = sorted(distances, key=compare)
    # extract only k closest labels
    k_labels = [label for (_, label) in by_distances[:k]]
    # return the majority voted label
    return find_majority(k_labels)

def new_predict_regr(k, train_pnts, train_labels, test_pnts):
    '''
    Predicts the new data-point's category/label by 
    looking at all other training labels
    '''
    # distances contains tuples of (distance, label)
    distances = [(euclidean_distance(test_pnts, pnt), label)
                    for (pnt, label) in zip(train_pnts, train_labels)]
    # sort the distances list by distances
    compare = lambda distance: distance[0]
    by_distances = sorted(distances, key=compare)
    # extract only k closest labels
    k_labels = [label for (_, label) in by_distances[:k]]
    # return the majority voted label
    return np.mean(k_labels)

In [ ]:
scores = []
scores3 = []
for part in range(10):
    print("Current part is %d\n" % part)
    train_x = np.concatenate((_train_x[0:(part*50)], _train_x[(part+1)*50:506]), axis=0)
    train_y = np.concatenate((_train_y[0:(part*50)], _train_y[(part+1)*50:506]), axis=0)
    test_x = _train_x[part*50:(part+1)*50]
    test_y = _train_y[part*50:(part+1)*50]
    scaler = preprocessing.StandardScaler().fit(train_x)
    train_x = scaler.transform(train_x)
    test_x = scaler.transform(test_x)
    transpose_train_x = np.transpose(train_x)
    
    cur_w = np.random.normal(0, 0.35, (number_of_neurons,1))
    cur_B =  np.random.normal(0, 0.35, (dimensionality, number_of_neurons))
    cur_biases = np.zeros((number_of_neurons))
    cur_P = np.zeros((dimensionality, dimensionality)) 
    cur_w_regr = np.random.normal(0, 0.35, (number_of_neurons_regr,1))
    cur_B_regr = np.random.normal(0, 0.35, (dimensionality, number_of_neurons_regr))
    cur_bias_regr = np.random.normal(0, 0.35, (number_of_neurons_regr))
    
    O = np.zeros((dimensionality, k)) 
    cur_iter = 0
    
    sess = tf.Session()
    init = tf.global_variables_initializer()
    sess.run(init, feed_dict={Dataplace: transpose_train_x})

    for epoch in range(num_epochs):
        print("Epoch %d" %(epoch))
        for iteration in range(num_iters):
            offset = (cur_iter % 12)*BATCH_SIZE
            sample_x = np.reshape(train_x[offset:(offset+BATCH_SIZE)], (BATCH_SIZE, dimensionality))
            sample_y = np.reshape(train_y[offset:(offset+BATCH_SIZE)], (BATCH_SIZE, 1))
            sess.run(target, feed_dict={Data: sample_x, outputs:sample_y, Lbd: Lambda, Dataplace: transpose_train_x,
                                        tf_data_w: cur_w,
                                        tf_data_B: cur_B,
                                        tf_data_P: cur_P,
                                        tf_w_regr: cur_w_regr,
                                        tf_B_regr: cur_B_regr,
                                        tf_bias_regr: cur_bias_regr})
            cur_iter = cur_iter+1
        reses = []
        for i in range(100):
            offset = (cur_iter % 12)*BATCH_SIZE
            sample_x = np.reshape(train_x[offset:(offset+BATCH_SIZE)], (BATCH_SIZE, dimensionality))
            sample_y = np.reshape(train_y[offset:(offset+BATCH_SIZE)], (BATCH_SIZE, 1))
            res = sess.run([loss_hybrid], feed_dict={Data: sample_x, outputs:sample_y, Lbd: Lambda, Dataplace: transpose_train_x, 
                                        tf_data_w: cur_w,
                                        tf_data_B: cur_B,
                                        tf_data_P: cur_P,
                                        tf_w_regr: cur_w_regr,
                                        tf_B_regr: cur_B_regr,
                                        tf_bias_regr: cur_bias_regr})
            cur_iter = cur_iter+1
            reses.append(res)
        print ("Iter %d: loss: %.4f\n" %(cur_iter, np.mean(np.array(reses))))
        lib.plot.plot('train cost', np.mean(np.array(reses)))
        lib.plot.tick()
        lib.plot.flush()

        cur_w, cur_B, cur_w_regr, cur_B_regr, cur_bias_regr = sess.run([w, B, w_regr, B_regr, bias_regr])

        third_grad_psi = np.reshape(sess.run([grad_psi]), (BATCH_SIZE_2, dimensionality))
        third_grad_psi_regr = np.reshape(sess.run([grad_psi_regr]), (BATCH_SIZE_2, dimensionality))
        for r in range(1000):
            sess.run([tf_data_second])
            np.concatenate((third_grad_psi, np.reshape(sess.run([grad_psi]), (BATCH_SIZE_2, dimensionality))), axis=0) 
            np.concatenate((third_grad_psi_regr, np.reshape(sess.run([grad_psi_regr]), (BATCH_SIZE_2, dimensionality))), axis=0)

        M = (1-rho)*np.matmul(np.transpose(third_grad_psi), third_grad_psi)+\
            rho*np.matmul(np.transpose(third_grad_psi_regr), third_grad_psi_regr)/var
        u, s, vh = np.linalg.svd(M, full_matrices=True)
        O = u[:,0:k:1]
        print(s)
        cur_P = np.matmul(O, np.transpose(O))
        tvr = 1-np.sum(np.multiply(s[0:k],s[0:k]))/np.sum(np.multiply(s,s))
        lib.plot.plot('total variance to retain', tvr)
        lib.plot.tick()
        lib.plot.flush()
    reduced_train_x = np.matmul(train_x, O)
    reduced_test_x = np.matmul(test_x, O)
    reg = LinearRegression().fit(reduced_train_x, train_y)
    score = reg.score(reduced_test_x, test_y)
    scores.append(score)
    print ("Part %d: score on test %.4f\n" %(part, score))
    color = [str(item/50.) for item in test_y]
    plt2.close()
    plt2.scatter(np.transpose(reduced_test_x)[0], np.transpose(reduced_test_x)[1], s=100, c=color)
    plt2.xlabel("1st component")
    plt2.ylabel("2nd component")
    plt2.show()
    sess.close()
    print(cur_P)

    test_y_prediction = []
    for test_image in reduced_test_x:
        pred = new_predict_regr(10, reduced_train_x, train_y, test_image)
        test_y_prediction.append(pred)
    score3 = 100*(1.0-np.mean((np.array(test_y_prediction)- test_y)**2)/np.var(test_y))
    scores3.append(score3)
    print('acc:', str(round(score3, 2))+'%')
print ("Average 10-NN acc on test %.4f\n" %(np.mean(np.array(scores3))))
print ("Average score on test %.4f\n" %(np.mean(np.array(scores))))

Current part is 0

Epoch 0
Iter 1100: loss: 3.3171

iter 1	train cost	3.31705904006958
[2.7557516  0.69724077 0.52576077 0.43784127 0.35854518 0.2871949
 0.26906607 0.20652054 0.1803683  0.1570237  0.1448512  0.12224548
 0.10088903]
iter 2	total variance to retain	0.09996926784515381
Epoch 1
Iter 2200: loss: 2.1292

iter 3	train cost	2.129249095916748
[3.3697047  0.9652594  0.5595607  0.4557453  0.36512104 0.3126747
 0.2946108  0.26110467 0.21535309 0.20989917 0.1790815  0.17342284
 0.13528855]
iter 4	total variance to retain	0.08064311742782593
Epoch 2
Iter 3300: loss: 1.4457

iter 5	train cost	1.4456514120101929
[4.4087386  1.1133327  0.75928223 0.70713055 0.5890588  0.5281674
 0.47656417 0.40317497 0.32581523 0.30558154 0.27084985 0.23714015
 0.1916863 ]
iter 6	total variance to retain	0.10624957084655762
Epoch 3
Iter 4400: loss: 1.0705

iter 7	train cost	1.0705299377441406
[4.849444   1.5880339  0.74230045 0.711669   0.63920057 0.59780866
 0.5165392  0.4705295  0.3991978  0.3563611

Iter 34100: loss: 0.2388

iter 61	train cost	0.2388288676738739
[6.4690123  1.2580606  0.08730327 0.06131354 0.03970566 0.03275031
 0.02934401 0.02706987 0.01964388 0.015383   0.01406629 0.01309104
 0.01081691]
iter 62	total variance to retain	0.0003851652145385742
Epoch 31
Iter 35200: loss: 0.2440

iter 63	train cost	0.2439848780632019
[7.4582453  1.806137   0.09847982 0.0653986  0.03043828 0.02864425
 0.02632103 0.02021304 0.01787254 0.01629215 0.01449614 0.0132751
 0.01055631]
iter 64	total variance to retain	0.0003040432929992676
Epoch 32
Iter 36300: loss: 0.2472

iter 65	train cost	0.24715310335159302
[7.9548516  1.8524617  0.12557341 0.05685673 0.03365954 0.03003976
 0.02574742 0.02411112 0.02098494 0.01954213 0.01562551 0.01235741
 0.01215565]
iter 66	total variance to retain	0.00035440921783447266
Epoch 33
Iter 37400: loss: 0.2410

iter 67	train cost	0.24100515246391296
[7.689691   1.4596567  0.09536027 0.04842052 0.03719857 0.02810878
 0.02530279 0.02160107 0.01955392 0.016967

Iter 64900: loss: 0.2057

iter 117	train cost	0.20574380457401276
[1.1109143e+01 2.8045831e+00 5.6254681e-02 4.4968296e-02 2.6758185e-02
 2.5006404e-02 2.1656385e-02 1.7564137e-02 1.5961064e-02 1.3434336e-02
 1.0419526e-02 8.7266229e-03 7.2110957e-03]
iter 118	total variance to retain	6.079673767089844e-05
Epoch 59
Iter 66000: loss: 0.2186

iter 119	train cost	0.2185775339603424
[1.1593894e+01 2.1990070e+00 4.3421365e-02 3.7589677e-02 2.4603305e-02
 2.3498628e-02 2.1674503e-02 1.9827034e-02 1.5169676e-02 1.5058966e-02
 1.0107766e-02 9.3027810e-03 6.4876233e-03]
iter 120	total variance to retain	4.315376281738281e-05
Epoch 60
